<a href="https://colab.research.google.com/github/pahwa1902/LearMachineLearning/blob/master/DisasterTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
from keras.layers import LSTM
from tensorflow.keras import layers
tf.random.set_random_seed(1)
import tensorflow_datasets as tfds
from gensim.models import KeyedVectors
tf.enable_eager_execution()

In [3]:
! ls '/content/drive/My Drive/nlp-getting-started'

sample_submission.csv  test.csv  train.csv


In [0]:
data_train = pd.read_csv('/content/drive/My Drive/nlp-getting-started/train.csv')

In [0]:
training_data = pd.DataFrame(data_train)

In [13]:
training_data.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [0]:
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\s)@\w+', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'#', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'http\S+', u'', x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'[?|$|.|!|=>|&|;|:)|+|@|*|-]',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'[0-9]',u'',x))
training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\w+):',u'',x))
#training_data['text'] = training_data['text'].apply(lambda x : re.sub(u'(\s+)',u'',x))

In [12]:
training_data.iloc[1190]['text']

'Leicester_Merc  ICYMI  Ashes  Australia collapse at Trent Bridge  how Twitter rea\x89Û_  '

In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
summ=[]
for i in range(len(training_data)):
  a = training_data['text'].iloc[i]
  summ+=a.split(' ')

In [82]:
vocab = len(set(summ))



ValueError: ignored

In [68]:
tokenizer = tfds.features.text.Tokenizer()
vocab_set = set()
for text_tensor,text_label in train_dataset:
  token = tokenizer.tokenize(text_tensor.numpy())
  vocab_set.update(token)
vocab_len = len(vocab_set)
print(vocab_len)

RuntimeError: ignored

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocab_set)

In [0]:
def encode(data,label):
  encoded_data = encoder.encode(data.numpy())
  return encoded_data, label

In [0]:
def encode_fn(text,label):
  return tf.py_function(encode,inp=[text,label],Tout=(tf.int32,tf.int32))

train_dataset = train_dataset.map(encode_fn)
validate_dataset = validate_dataset.map(encode_fn)

In [0]:
label = training_data.pop('target')
train_data = training_data.iloc[:5700]['text']
train_label = label[:5700]
validate_data = training_data.iloc[5700:]['text']
validate_label = label[5700:]

In [79]:
filename = 'GoogleNews-vectors-negative300.bin.gz.'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data,train_label))
train_dataset = train_dataset.shuffle(len(train_data)).batch(64)
validate_dataset = tf.data.Dataset.from_tensor_slices((validate_data,validate_label))


In [0]:
model = tf.keras.Sequential();
model.add(tf.keras.layers.Embedding(vocab,20));
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)));
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [51]:
model.fit(train_dataset,epochs=10,verbose=1,validation_data=validate_dataset)

ValueError: ignored